## Create reference evapotranspiration time series (FLUXCOM)

Unit Conversion is based on: https://earthscience.stackexchange.com/questions/20733/fluxnet15-how-to-convert-latent-heat-flux-to-actual-evapotranspiration

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
from glob import glob
from pathlib import Path

import os
import iris
import xarray as xr
import pandas as pd
from esmvalcore.preprocessor import regrid
from pathos.threading import ThreadPool as Pool
from dask.diagnostics import ProgressBar

## Set Paths

In [ ]:
# Set Paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/model_refinement_pub/')
AUXDIR = Path(f"{ROOT}/aux_data/")
OBSDIR = Path(f"{ROOT}/observations/")
MODELS = Path(f'{ROOT}/model_parameters/wflow_sbm/')
OUTPUT = Path(f'{OBSDIR}/evaporation/regridded_FLUXCOM/')

## Config

In [ ]:
# Get available basin IDs wflow_sbm
basin_dirs = glob(f'{MODELS}/*')
basin_ids = [s.split('/')[-1] for s in basin_dirs]
basin_ids.sort()

# Amount of available cores
cores_available = 1

## Preprocess function

In [ ]:
def prep_observations(basin_id):
    print(basin_id)
    # Set basin directory
    BASINDIR = f'{MODELS}/{basin_id}/'
    
    # Open netCDF file as an example grid from the model directory
    cube_example = iris.load(f'{BASINDIR}/staticmaps.nc')[1]

    # Guess bounds   
    cube_example.coord('y').guess_bounds()
    cube_example.coord('x').guess_bounds()

    # Rename Coords
    cube_example.coord('y').rename('latitude')
    cube_example.coord('x').rename('longitude')

    cube_example.coord('latitude').units = 'degrees'
    cube_example.coord('longitude').units = 'degrees'
    
    # Load observation netCDF files
    files = glob(f'{OBSDIR}/evaporation/FLUXCOM/*.nc')
    
    def edit_attributes(cube, field, filename):
        cube.attributes.pop('History', 'none')

    cubes_obs = iris.load(files, 'latent heat', callback=edit_attributes)
    cube_obs = cubes_obs.concatenate_cube()
    
    # Regrid observation cube
    cube_out = regrid(cube_obs, cube_example, scheme='area_weighted')
    
    # Create obs dataset
    da = xr.DataArray.from_iris(cube_out)

    # Create mask dataset
    mask = xr.open_dataset(f'{BASINDIR}/staticmaps.nc').mask
    mask = mask.rename({'y':'latitude','x':'longitude'})

    # Convert unit to mm/time
    da = da / 2.45

    # Apply mask
    da = da.where(mask>0)
    
    # Calculate time series
    da = da.mean(['latitude','longitude'])
    da = da.chunk(chunks='auto')
    da = da.drop('spatial_ref')
    
    # Output filename
    output_fname = f'{OUTPUT}/{basin_id}_FLUXCOM_evaporation_ref_2008_2015.nc'

    # Save to netcdf
    write_job = da.to_netcdf(output_fname, compute=False)
    with ProgressBar():
        write_job.compute()
        
    return print(f'{basin_id} finished: {output_fname}')

## Parallel Run Function

In [ ]:
def parallel_run(
    basin_ids,
    threads=cores_available,
    ):
    
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
    # Run parallel models
    pool.map(
        prep_observations,
        basin_ids,
        )
    return

## Sort basins by size

In [ ]:
# Sort by basin size
def sort_basin_ids_by_size(basin_ids):
    sizes = []
    for basin_id in basin_ids:
        size = os.path.getsize(f'{MODELS}/{basin_id}/staticmaps.nc')
        sizes.append(size)

    df = pd.DataFrame()
    df['basin_id'] = basin_ids
    df['size'] = sizes
    df = df.sort_values('size')

    basin_ids = df.basin_id.to_list()
    
    return basin_ids

basin_ids_sorted = sort_basin_ids_by_size(basin_ids)

# Run parallel function

In [ ]:
# Run function
parallel_run(basin_ids_sorted)